In [15]:
import numpy as np
import scipy.integrate
scipy.integrate.trapz = np.trapz
import yaml
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, PreText, DataTable, Select
from bokeh.plotting import figure, curdoc
from bokeh.themes import Theme

from pytmatrix.tmatrix import Scatterer
from pytmatrix.psd import PSDIntegrator, GammaPSD
from pytmatrix import orientation, radar, tmatrix_aux, refractive

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
from scipy.special import gamma


In [16]:
# calculations
def drop_ar(D_eq):
    if D_eq < 0.7:
        return 1.0;
    elif D_eq < 1.5:
        return 1.173 - 0.5165*D_eq + 0.4698*D_eq**2 - 0.1317*D_eq**3 - \
            8.5e-3*D_eq**4
    else:
        return 1.065 - 6.25e-2*D_eq - 3.99e-3*D_eq**2 + 7.66e-4*D_eq**3 - \
            4.095e-5*D_eq**4 


def get_scattering_props(Dm=2.0,logNw=3.0,mu=0.0,wavelength='10',cant=0.0,ptype='rain'):
    if ((wavelength == '5') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_C, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '3') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_X, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '10') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_S, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '5') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_C, m=refractive.m_w_10C[tmatrix_aux.wl_C])
    elif ((wavelength == '3') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_X, m=refractive.m_w_10C[tmatrix_aux.wl_X])
    elif ((wavelength == '10') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_S, m=refractive.m_w_10C[tmatrix_aux.wl_S])
    scatterer.psd_integrator = PSDIntegrator()
    if ptype == 'rain':
        scatterer.psd_integrator.axis_ratio_func = lambda D: 1.0/drop_ar(D)
    if ptype == 'hail':
        scatterer.psd_integrator.axis_ratio_func = lambda D: 0.99
    scatterer.psd_integrator.D_max = 10.0
    scatterer.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)
    if cant > 0.0:
        scatterer.or_pdf = orientation.gaussian_pdf(cant)
    scatterer.orient = orientation.orient_averaged_fixed
    scatterer.psd_integrator.init_scatter_table(scatterer)

    D0 = (3.67 + mu)/(4 + mu) * Dm
    Nw = 10.**logNw   
    scatterer.psd = GammaPSD(D0=D0, Nw=Nw, mu=mu)

    Zh = 10*np.log10(radar.refl(scatterer))
    Zv = 10*np.log10(radar.refl(scatterer, False))
    Zdr = 10.*np.log10(radar.Zdr(scatterer))
    Ldr = 10*np.log10(radar.ldr(scatterer))
    rho_hv = radar.rho_hv(scatterer)
    delta = radar.delta_hv(scatterer)
    scatterer.set_geometry(tmatrix_aux.geom_horiz_forw)
    Kdp = radar.Kdp(scatterer)
    Ah = radar.Ai(scatterer)
    Av = radar.Ai(scatterer,h_pol=False)

    f_u = (6/(4**4))*((4 + mu)**(mu + 4))/(gamma(mu+4))

    dsd_df = calc_dsd(Dm,logNw,mu)

    NT = Nw * f_u * gamma(mu + 1) * Dm / ((4 + mu)**(mu + 1))
    if ptype == 'rain':
        LWC = (1. * np.pi * Nw * Dm**4) / (4.**4 * 1000.)
    elif ptype == 'hail':
        LWC = (1. * np.pi * Nw * Dm**4) / (4.**4 * 917.)
        
#    Vt = -0.193 + 4.96*dsd_df['D'] - 0.904*dsd_df['D']**2 + 0.0566*dsd_df['D']**3
    
    Vt = 17.67 * dsd_df['D']**0.67
    
    R = np.sum(dsd_df['ND']*(np.pi*dsd_df['D']/6.)*Vt)/3600.



    integ_df = pd.DataFrame({'NT':[NT], 
                            'WC (g m-3)':[LWC], 
                            'Zh (dBZ)':[Zh], 
                            'Zv (dBZ)':[Zv], 
                            'Zdr (dB)': [Zdr], 
                            'Ldr (dB)': [Ldr], 
                            'rho_hv':[rho_hv], 
                            'Kdp (deg km-1)':[Kdp], 
                            'delta (deg km-1)':[delta],
                            'Ah (dBZ/km)':[Ah],
                            'Adr (dB/km)':[Ah-Av],
                            'R (mm/hr)': R})

    return dsd_df, integ_df



def calc_dsd(Dm,logNw,mu):
    f_u = (6/4**4)*((4 + mu)**(mu + 4))/(gamma(mu+4))
    dsd_df = pd.DataFrame()
    dsd_df['D'] = np.arange(0.1,20,0.1)
    dsd_df['ND'] = 10**logNw * f_u * ((dsd_df['D'] / Dm) ** mu) * np.exp(-1.*(4 + mu)*(dsd_df['D'] / Dm))
 
    return dsd_df


In [17]:
# Set up data
Dm = 1.5
logNw = np.log10(5800)
mu = 3
wavelength = '10'
cant = 0.0
ptype = 'rain'

dsd_df, integ_df = get_scattering_props(Dm=Dm, logNw=logNw, mu=mu, wavelength=wavelength, 
    cant=cant, ptype=ptype)

In [18]:
integ_df

,NT,WC (g m-3),Zh (dBZ),Zv (dBZ),Zdr (dB),Ldr (dB),rho_hv,Kdp (deg km-1),delta (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,582.832,0.360,36.161,35.397,0.764,-35.282,0.999,0.086,0.001,0.003,0.000,13.156


In [19]:
!wget https://github.com/maahn/pyOptimalEstimation_examples/raw/master/data/huntsville_parameters.nc

--2025-11-06 04:28:08--  https://github.com/maahn/pyOptimalEstimation_examples/raw/master/data/huntsville_parameters.nc
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/maahn/pyOptimalEstimation_examples/master/data/huntsville_parameters.nc [following]
--2025-11-06 04:28:08--  https://raw.githubusercontent.com/maahn/pyOptimalEstimation_examples/master/data/huntsville_parameters.nc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918986 (897K) [application/octet-stream]
Saving to: ‘huntsville_parameters.nc.2’

huntsville_paramete 100%[===================>] 897.45K  --.-KB/s    in 0.02s   

2025-11-

In [20]:
import xarray as xr
data = xr.open_dataset('huntsville_parameters.nc', engine='netcdf4')

In [21]:
data.Nw.max

<bound method DataArrayAggregations.max of <xarray.DataArray 'Nw' (time: 18969)> Size: 152kB
[18969 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 152kB 2009-12-13T04:24:00 ... 2011-04-04T2...
Attributes:
    units:    1/m3/mm>

In [22]:
def calc_dsd_all(Dm,Nw):
    dsd_df, integ_df = get_scattering_props(Dm=float(data.Dm[i]), logNw=float(np.log10(data.Nw[i])), 
                                            mu=np.random.normal(3,0.5), wavelength=wavelength, 
                                            cant=cant, ptype=ptype)
    return integ_df

In [23]:
from dask.distributed import Client
client = Client() 

/home/tob3/.conda/envs/logreg-weather-py311/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39171 instead
  warnings.warn(


2025-11-06 04:28:16,065 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:33939'.
2025-11-06 04:28:16,087 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:36887'.
2025-11-06 04:28:16,380 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:40455'.
2025-11-06 04:28:16,685 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:37515'.


In [24]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39171/status,
Dashboard: http://127.0.0.1:39171/status,Workers: 4
Total threads: 4,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44003,Workers: 0
Dashboard: http://127.0.0.1:39171/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:36887,Total threads: 1
Dashboard: http://127.0.0.1:39047/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:45757,


In [25]:
import numpy as np
futures = client.map(calc_dsd_all,data.Dm.values,np.log10(data.Nw.values))

In [26]:
futures

[<Future: pending, key: calc_dsd_all-e15ff44454d60b6fda2f607b5673a52f>,
 <Future: pending, key: calc_dsd_all-7c70343bc08eebf0821778c58d0b102a>,
 <Future: pending, key: calc_dsd_all-7c71786c70150b8633c1c667445c0571>,
 <Future: pending, key: calc_dsd_all-9d722d667c2a883503ee25cb0535abea>,
 <Future: pending, key: calc_dsd_all-ee5d98756c4297c2f7096b9cf600a157>,
 <Future: pending, key: calc_dsd_all-35f7fbf7a027f6470de23d165a8784a7>,
 <Future: pending, key: calc_dsd_all-5520ef7cb2b3bba56611f0d2dd012349>,
 <Future: pending, key: calc_dsd_all-38d77294922fc738fa1ed0884184b85a>,
 <Future: pending, key: calc_dsd_all-45606642439c7652209b4f554affd902>,
 <Future: pending, key: calc_dsd_all-016c572646f668afb45f707f524df9b4>,
 <Future: pending, key: calc_dsd_all-998f3ad30dba2c95ecfd030e357b7903>,
 <Future: pending, key: calc_dsd_all-b317b83bdddd98c12c5fed51a6aae761>,
 <Future: pending, key: calc_dsd_all-b28b6143b79a8202e6f295fc7c2ecf75>,
 <Future: pending, key: calc_dsd_all-075b46a6ceefab6ded30b8de679

In [29]:
import os

output_dir = os.path.expanduser("~/Downloads/out")
os.makedirs(output_dir, exist_ok=True)

In [31]:
import pandas as pd

all_df = pd.DataFrame()
for i in range(data.time.size):
    print(i)
    dsd_df, integ_df = get_scattering_props(Dm=float(data.Dm[i]), logNw=float(np.log10(data.Nw[i])), 
                                            mu=np.random.normal(3,0.5), wavelength=wavelength, 
                                            cant=cant, ptype=ptype)
    integ_df.to_csv('~/Downloads/out/scatt_{:05d}.csv'.format(i))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


KeyboardInterrupt: 

In [ ]:
float(data.Dm[i])

2025-11-06 04:26:35,657 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-7744cfc80d9274edeb8cf17a90c6629d
State:     executing
Task:  <Task 'calc_dsd_all-7744cfc80d9274edeb8cf17a90c6629d' calc_dsd_all(...)>
Exception: 'NameError("name \'get_scattering_props\' is not defined")'
Traceback: '  File "/tmp/ipykernel_30620/952345075.py", line 2, in calc_dsd_all\n'

2025-11-06 04:26:35,659 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-62bfc3a7fae45c2bc457af59de7a6602
State:     executing
Task:  <Task 'calc_dsd_all-62bfc3a7fae45c2bc457af59de7a6602' calc_dsd_all(...)>
Exception: 'NameError("name \'get_scattering_props\' is not defined")'
Traceback: '  File "/tmp/ipykernel_30620/952345075.py", line 2, in calc_dsd_all\n'

2025-11-06 04:26:35,662 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-3a98c8e2ed2e5265753419e73d037a8b
State:     executing
Task:  <Task 'calc_dsd_all-3a98c8e2ed2e5265753419e73d037a8b' calc_dsd_all(...)>
Except

NameError: name 'i' is not defined

2025-11-06 04:26:35,689 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-c530b3432c7289bdbec9a72d1ab72618
State:     executing
Task:  <Task 'calc_dsd_all-c530b3432c7289bdbec9a72d1ab72618' calc_dsd_all(...)>
Exception: 'NameError("name \'get_scattering_props\' is not defined")'
Traceback: '  File "/tmp/ipykernel_30620/952345075.py", line 2, in calc_dsd_all\n'

2025-11-06 04:26:35,691 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-7fa3e35ddb64c95d55f9f812807dee81
State:     executing
Task:  <Task 'calc_dsd_all-7fa3e35ddb64c95d55f9f812807dee81' calc_dsd_all(...)>
Exception: 'NameError("name \'get_scattering_props\' is not defined")'
Traceback: '  File "/tmp/ipykernel_30620/952345075.py", line 2, in calc_dsd_all\n'



2025-11-06 04:26:35,694 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-9a66b4069ae4fa043ec9495ebf73fcdf
State:     executing
Task:  <Task 'calc_dsd_all-9a66b4069ae4fa043ec9495ebf73fcdf' calc_dsd_all(...)>
Exception: 'NameError("name \'get_scattering_props\' is not defined")'
Traceback: '  File "/tmp/ipykernel_30620/952345075.py", line 2, in calc_dsd_all\n'

2025-11-06 04:26:35,695 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-5ef5abf8e038e4e7378a3181c4c72b25
State:     executing
Task:  <Task 'calc_dsd_all-5ef5abf8e038e4e7378a3181c4c72b25' calc_dsd_all(...)>
Exception: 'NameError("name \'get_scattering_props\' is not defined")'
Traceback: '  File "/tmp/ipykernel_30620/952345075.py", line 2, in calc_dsd_all\n'

2025-11-06 04:26:35,701 - distributed.worker - ERROR - Compute Failed
Key:       calc_dsd_all-f312d894bea2223618975963832a543d
State:     executing
Task:  <Task 'calc_dsd_all-f312d894bea2223618975963832a543d' calc_dsd_all(...)>
Except